### Ingestion File Movie Cast

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment =  dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date =  dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()

movie_cast_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("personId", IntegerType(), True),
    StructField("characterName", StringType(), True),
    StructField("genderId", IntegerType(), True),
    StructField("castOrder", IntegerType(), True),
])

# Se va a leer un archivo JSON multilinea, por defecto la opcion "multiLine" es False
movie_cast = spark\
    .read\
    .schema(movie_cast_schema)\
    .option("multiLine", True)\
    .json(f"{bronze_folder_path}/{v_file_date}/movie_cast.json")

movie_cast_df = movie_cast\
    .withColumnRenamed("movieId", "movie_id")\
    .withColumnRenamed("personId", "person_id")\
    .withColumnRenamed("characterName", "character_name")\
    .withColumn("ingestion_date", current_timestamp())\
    .withColumn("environment", lit("PROD"))\
    .withColumn("file_date", lit(v_file_date))\
    .drop("genderId", "castOrder")

# overwrite_partition(movie_cast_df, "movie_silver", "movie_cast", "file_date")


merge_condition = "tgt.movie_id = src.movie_id AND tgt.person_id = src.person_id AND tgt.file_date = src.file_date"
merge_delta_lake(movie_cast_df, "movie_silver", "movie_cast", silver_folder_path, merge_condition, "file_date")

# movie_cast_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movie_cast")


In [0]:
%sql
select file_date, count(1) 
from movie_silver.movie_cast
group by file_date;

In [0]:
%sql
select *
from movie_silver.movie_cast;

In [0]:
dbutils.notebook.exit("Success")